<a href="https://colab.research.google.com/github/fjme95/python-para-la-ciencia-de-datos/blob/main/Semana%206/Detecci%C3%B3n_duplicados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencias

In [20]:
!pip install xgboost

In [21]:
!pip install -U gensim

In [22]:
import zipfile

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report

from gensim.utils import tokenize as gensim_tokenize
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

import nltk
from nltk.corpus import stopwords

from xgboost import XGBClassifier

In [23]:
nltk.download('stopwords')

stopwords_list = stopwords.words('english') + ['would']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Datos

Vamos a ocupar el dataset "[Questions Pairs Dataset](https://www.kaggle.com/quora/question-pairs-dataset)" de Quora. Consiste en 404,351 pares de preguntas con una etiqueta que indica si ambas preguntas son lo mismo.

Por ejemplo, el par de preguntas 

- Do you believe there is life after death?
- Is it true that there is life after death?

se refieren a lo mismo, por lo que estarán marcadas como duplicadas.

## Descarga de datos

In [24]:
!wget https://github.com/fjme95/python-para-la-ciencia-de-datos/blob/main/Semana%206/data/questions.csv.zip?raw=true

--2021-11-25 21:09:39--  https://github.com/fjme95/python-para-la-ciencia-de-datos/blob/main/Semana%206/data/questions.csv.zip?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/fjme95/python-para-la-ciencia-de-datos/raw/main/Semana%206/data/questions.csv.zip [following]
--2021-11-25 21:09:40--  https://github.com/fjme95/python-para-la-ciencia-de-datos/raw/main/Semana%206/data/questions.csv.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/fjme95/python-para-la-ciencia-de-datos/main/Semana%206/data/questions.csv.zip [following]
--2021-11-25 21:09:40--  https://raw.githubusercontent.com/fjme95/python-para-la-ciencia-de-datos/main/Semana%206/data/questions.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.10

In [25]:
with zipfile.ZipFile('/content/questions.csv.zip?raw=true', 'r') as zip_ref:
    zip_ref.extractall()

## Lectura de datos 

In [26]:
questions_raw = pd.read_csv('questions.csv').set_index('id')
questions_raw

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...
404346,789792,789793,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404347,789794,789795,Do you believe there is life after death?,Is it true that there is life after death?,1
404348,789796,789797,What is one coin?,What's this coin?,0


## Análisis de los datos

In [27]:
questions_raw.is_duplicate.value_counts() / len(questions_raw)

0    0.630752
1    0.369248
Name: is_duplicate, dtype: float64

In [28]:
questions_raw.iloc[:, -1:]

,is_duplicate
id,
0,0
1,0
2,0
3,0
4,0
...,...
404346,0
404347,1
404348,0


## División en train y test

In [29]:
X_train, X_test, y_train, y_test = train_test_split(questions_raw.iloc[:, :-1], questions_raw.iloc[:, -1:], random_state=10)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(303263, 4) (101088, 4) (303263, 1) (101088, 1)


In [30]:
X_train

,qid1,qid2,question1,question2
id,,,,
57436,114273,114274,"Which job title has a higher salary, front-end...",How hard is it to switch from being a front en...
217293,428435,428436,Did Christianity start with Jesus?,Do Muslims follow Jesus more than Christians do?
43481,86595,86596,What is swarm intelligence and how does it works?,How does swarm intelligence work?
224844,443154,443155,What are the practical steps in a statutory au...,What work does statutory auditor do?
194743,384393,384394,What is the difference between usage of can an...,What is the difference between the usage of do...
...,...,...,...,...
105595,209444,209445,What would happen if you inject/infect a tumou...,"What happens when a virus, bacterium etc inter..."
93553,185728,185729,How can I improve my writing skills and blog s...,How do I improve my writing skills?
356879,698900,698901,What does it mean when you have low self esteem?,"Why do I think like I have high self-esteem, b..."


## Procesamiento

In [31]:
def tokenize(text):
    return [word for word in gensim_tokenize(str(text), deacc = True, lower = True) if word not in stopwords_list]

In [32]:
index = np.concatenate([X_train.qid1, X_train.qid2])
X_train_long = np.concatenate([X_train.question1, X_train.question2])

# Doc2Vec

In [33]:
documents = [TaggedDocument(tokenize(doc), [f'q{i}']) for doc, i in zip(X_train_long, index)]
documents

[TaggedDocument(words=['job', 'title', 'higher', 'salary', 'front', 'end', 'developer', 'back', 'end', 'developer'], tags=['q114273']),
 TaggedDocument(words=['christianity', 'start', 'jesus'], tags=['q428435']),
 TaggedDocument(words=['swarm', 'intelligence', 'works'], tags=['q86595']),
 TaggedDocument(words=['practical', 'steps', 'statutory', 'audit'], tags=['q443154']),
 TaggedDocument(words=['difference', 'usage', 'could'], tags=['q384393']),
 TaggedDocument(words=['treat', 'burning', 'dry', 'lips'], tags=['q87623']),
 TaggedDocument(words=['first', 'thing', 'comes', 'mind', 'hear', 'word', 'filipino'], tags=['q142944']),
 TaggedDocument(words=['aspire', 'great', 'power', 'third', 'rate', 'economy'], tags=['q230244']),
 TaggedDocument(words=['qualification', 'get', 'job', 'fox', 'traveller', 'national', 'geography'], tags=['q637610']),
 TaggedDocument(words=['differences', 'mp', 'file', 'flac', 'file', 'converted'], tags=['q315145']),
 TaggedDocument(words=['could', 'self', 'learn'

In [34]:
model = Doc2Vec(documents, vector_size = 200, window = 2)

In [35]:
model.save('doc2vec_200.model')

In [43]:
model.dv.most_similar('q142866')

[('q399702', 0.8708192706108093),
 ('q142867', 0.8597369194030762),
 ('q550435', 0.8402022123336792),
 ('q337119', 0.8379374146461487),
 ('q728802', 0.8224001526832581),
 ('q72079', 0.8218854665756226),
 ('q179375', 0.821225643157959),
 ('q5976', 0.8154377341270447),
 ('q27196', 0.8143490552902222),
 ('q157282', 0.808194637298584)]

In [44]:
def get_question_by_index(index):
    partial_res = questions_raw[questions_raw.qid2 == index]
    if len(partial_res) > 0:
        return partial_res.question2.values[0]
    else:
        return questions_raw[questions_raw.qid1 == index].question1.values[0]

def print_similar_questions(question_index):
    print(f'Pregunta:\n{get_question_by_index(question_index)}')

    for index, score in model.dv.most_similar(f'q{question_index}'):
        aux_index = int(index.replace('q', ''))
        print(aux_index)
        print(score, get_question_by_index(aux_index))

In [50]:
print_similar_questions(30)

Pregunta:
What are the laws to change your status from a student visa to a green card in the US? How do they compare to the immigration laws in Japan?
563703
0.9256182909011841 What are the laws to change your status from a student visa to a green card in the US, how do they compare to the immigration laws in Canada?
784874
0.9116592407226562 What are the laws to change your status from a student visa to a green card in the US? How do they compare to the immigration laws in Japan?
217105
0.9109957814216614 What are the laws to change your status from a student visa to a green card in the US, how do they compare to the immigration laws in Scottland?
217106
0.9092130661010742 What are the laws to change your status from a student visa to a green card in the US? How do they compare to the immigration laws in Japan?
784873
0.8971035480499268 What are the laws to change your status from a student visa to a green card in the US? How do they compare to the immigration laws in Bulgaria?
29
0.8

# Clasificación - Boosting

In [ ]:
q1_to_vect = [model.dv.get_vector(f'q{i}') for i in X_train['qid1'].values]
q2_to_vect = [model.dv.get_vector(f'q{i}') for i in X_train['qid2'].values]

In [ ]:
X_train_clf = np.hstack([q1_to_vect, q2_to_vect])
X_train_clf.shape

## XGBoost

### Entrenamiento

In [ ]:
xgb_clf = XGBClassifier()
eval_set = [(X_train_clf, y_train)]
xgb_clf.fit(X_train_clf, y_train.is_duplicate.ravel(), early_stopping_rounds=10, eval_metric="logloss", eval_set=eval_set, verbose=True)

### Evaluación en entrenamiento

In [ ]:
print(classification_report(y_train.is_duplicate.ravel(), xgb_clf.predict(X_train_clf)))

## AdaBoost

### Entrenamiento y evaluación

In [ ]:
ada_clf = AdaBoostClassifier()
ada_train_pred = ada_clf.fit(X_train_clf, y_train.is_duplicate.ravel()).predict(X_train_clf)
print(classification_report(y_train.is_duplicate.ravel(), ada_train_pred))

## Procesamiento del set de prueba

In [ ]:
q1_to_vect = [model.infer_vector(tokenize(text)) for text in X_test['question1'].values]
q2_to_vect = [model.infer_vector(tokenize(text)) for text in X_test['question2'].values]

In [ ]:
X_test_clf = np.hstack([q1_to_vect, q2_to_vect])
X_test_clf.shape

### Evaluación

In [ ]:
xgb_test_pred = xgb_clf.predict(X_test_clf)

In [ ]:
ada_test_pred = ada_clf.predict(X_test_clf)

In [ ]:
print(classification_report(y_test, xgb_test_pred))
print(classification_report(y_test, ada_test_pred))

# Ejercicios


- Se hizo la vectorización de los textos con Doc2Vec, intentar usando CountVectorizer y TfIdfVectorizer

- Cambiar el clasificador con cualquiera de los otros vistos (Linear Discriminant Analysis y Regresión Logística)

## Extra

- Ocupar GridSearch para encontrar los hiperparámetros de los modelos de clasificación
- Usando CountVectorizer o TfidfVectorizer, usar ```sklearn.pipeline.Pipeline``` para crear una "tubería" de datos que facilite el uso del vectorizador y el modelo.